In [16]:
import pandas as pd
import re

In [31]:
# Login using e.g. `huggingface-cli login` to access this dataset
df = pd.read_csv("hf://datasets/stasvinokur/cve-and-cwe-dataset-1999-2025/CVE_CWE_2025.csv")
print(df.head())
print(f"Dataset contains {len(df)} records.")
print(df.columns)

   ID         CVE-ID  CVSS-V4  CVSS-V3  CVSS-V2 SEVERITY  \
0   1  CVE-1999-0001      NaN      NaN      5.0   MEDIUM   
1   2  CVE-1999-0002      NaN      NaN     10.0     HIGH   
2   3  CVE-1999-0003      NaN      NaN     10.0     HIGH   
3   4  CVE-1999-0004      NaN      NaN      5.0   MEDIUM   
4   5  CVE-1999-0005      NaN      NaN     10.0     HIGH   

                                         DESCRIPTION         CWE-ID  
0  ip_input.c in BSD-derived TCP/IP implementatio...         CWE-20  
1  Buffer overflow in NFS mountd gives root acces...        CWE-119  
2  Execute commands as root via buffer overflow i...  NVD-CWE-Other  
3  MIME buffer overflow in email clients, e.g. So...  NVD-CWE-Other  
4  Arbitrary command execution via IMAP buffer ov...  NVD-CWE-Other  
Dataset contains 280694 records.
Index(['ID', 'CVE-ID', 'CVSS-V4', 'CVSS-V3', 'CVSS-V2', 'SEVERITY',
       'DESCRIPTION', 'CWE-ID'],
      dtype='object')


In [32]:
df['CVSS'] = df['CVSS-V4'].fillna(df['CVSS-V3'])

# Nettoyage CWE
df['CWE-ID'] = df['CWE-ID'].replace('NVD-CWE-Other', 'Unknown')

# Vérifier valeurs manquantes
print(df[['CVSS', 'CWE-ID', 'SEVERITY']].isnull().sum())

CVSS        74621
CWE-ID          0
SEVERITY     2104
dtype: int64


In [33]:
# Combler SEVERITY manquante en utilisant CVSS
def severity_from_cvss(cvss):
    if pd.isnull(cvss):
        return 'Unknown'
    elif cvss < 4:
        return 'Low'
    elif cvss < 7:
        return 'Medium'
    else:
        return 'High'

In [34]:
df['SEVERITY'] = df['SEVERITY'].fillna(df['CVSS'].apply(severity_from_cvss))

In [35]:
# Définir le niveau de risque (Risk_Level) basé sur CVSS
def risk_level(cvss):
    if pd.isnull(cvss):
        return 'Unknown'
    elif cvss < 4:
        return 'Low'
    elif cvss < 7:
        return 'Medium'
    else:
        return 'High'

In [36]:
df['Risk_Level'] = df['CVSS'].apply(risk_level)


In [37]:
print(df[['CVE-ID', 'CVSS', 'SEVERITY', 'CWE-ID', 'Risk_Level']].head())

          CVE-ID  CVSS SEVERITY   CWE-ID Risk_Level
0  CVE-1999-0001   NaN   MEDIUM   CWE-20    Unknown
1  CVE-1999-0002   NaN     HIGH  CWE-119    Unknown
2  CVE-1999-0003   NaN     HIGH  Unknown    Unknown
3  CVE-1999-0004   NaN   MEDIUM  Unknown    Unknown
4  CVE-1999-0005   NaN     HIGH  Unknown    Unknown


In [38]:
risk_summary = df['Risk_Level'].value_counts()

In [39]:
print("\nNombre de vulnérabilités par niveau de risque :")
print(risk_summary)


Nombre de vulnérabilités par niveau de risque :
Risk_Level
High       110759
Medium      90727
Unknown     74621
Low          4587
Name: count, dtype: int64


In [40]:
df['DESCRIPTION'] = df['DESCRIPTION'].astype(str).str.strip()

In [ ]:
# Liste de produits connus (améliore la précision)
KNOWN_PRODUCTS = [
    "Windows", "BIND", "IMAP", "POP", "FTP", "SSH", "ssh-agent",
    "rpc.statd", "statd", "NIS", "NFS", "Tooltalk", "TCP/IP",
    "Teardrop", "Land", "PKCS"
]

# Mots à ignorer
BLACKLIST = [
    "buffer", "overflow", "command", "execution", "remote", "local",
    "vulnerability", "denial", "service", "root", "access", "arbitrary",
    "privileged", "sessions", "encrypted", "gives", "allows", "gives",
    "cause", "can", "than", "other", "allow"
]

def clean_product(p):
    if not p:
        return None
    p = p.strip().replace(".", " ").replace(",", " ").replace("  ", " ")

    # enlever 'and', 'other than', etc.
    p = re.sub(r'\band\b|\bother\b|\bthan\b', '', p, flags=re.I)

    # supprimer les numéros de versions
    p = re.sub(r'\d+(\.\d+)*', '', p)

    # nettoyer blacklist
    tokens = [t for t in p.split() if t.lower() not in BLACKLIST]
    p = " ".join(tokens).strip()

    if len(p.split()) > 3:
        p = " ".join(p.split()[:3])  # limiter longueur

    return p if p else None


def extract_product_v4(text):
    text = text.strip()

    # 1. Produits connus présents dans la description 
    for prod in KNOWN_PRODUCTS:
        if prod.lower() in text.lower():
            return prod

    # 2. modèle : "in X"
    m = re.search(r'in ([A-Za-z0-9\-\._ ]+)', text)
    if m:
        p = clean_product(m.group(1))
        if p:
            return p

    # 3. modèle : "via X"
    m = re.search(r'via ([A-Za-z0-9\-\._ ]+)', text)
    if m:
        p = clean_product(m.group(1))
        if p:
            return p

    # 4. modèle : "on X"
    m = re.search(r'on ([A-Za-z0-9\-\._ ]+)', text)
    if m:
        p = clean_product(m.group(1))
        if p:
            return p

    # 5. modèle spécial : "X servers"
    m = re.search(r'([A-Za-z0-9\-]+) servers', text)
    if m:
        return clean_product(m.group(1) + " servers")

    # 6. modèle : "X clients"
    m = re.search(r'([A-Za-z0-9\-]+) clients', text)
    if m:
        return clean_product(m.group(1) + " clients")

    # 7. modèle : attaques connues (Teardrop, Land)
    if "Teardrop" in text:
        return "Windows IP stack"
    if "Land" in text:
        return "Windows TCP/IP"

    return "Unknown"


In [49]:
df["PRODUCT_EXTRACTED"] = df["DESCRIPTION"].apply(extract_product_v4)

# Afficher un aperçu
print(df[["PRODUCT_EXTRACTED"]].head(30))

      PRODUCT_EXTRACTED
0                TCP/IP
1                   NFS
2              Tooltalk
3         email clients
4                  IMAP
5                   POP
6                  PKCS
7                   NIS
8                  BIND
9                  BIND
10                 BIND
11              Windows
12                  SSH
13                  CDE
14             Teardrop
15                 Land
16                  FTP
17                statd
18            rpc.statd
19            Count cgi
20                rdist
21                rdist
22                 BIND
23            df on SGI
24          pset on SGI
25         eject on SGI
26                login
27        ordist on SGI
28         xlock on SGI
29  Internet Explorer x


In [61]:
severity_to_score = {
    "CRITICAL": 4,  
    "HIGH": 3,
    "MEDIUM": 2,
    "LOW": 1,
    "UNKNOWN": 0
}

# Créer une nouvelle colonne Risk_Score
df["Risk_Score"] = df["SEVERITY"].map(severity_to_score)

In [62]:
print(df[["CVE-ID", "SEVERITY", "Risk_Score"]].head())

          CVE-ID  SEVERITY  Risk_Score
0  CVE-1999-0001    MEDIUM         2.0
1  CVE-1999-0002      HIGH         3.0
4  CVE-1999-0005      HIGH         3.0
5  CVE-1999-0006  CRITICAL         4.0
6  CVE-1999-0007    MEDIUM         2.0


In [63]:
# Somme des scores par produit
risk_by_product = df.groupby("PRODUCT_EXTRACTED")["Risk_Score"].sum().sort_values(ascending=False)

In [64]:
count_by_product = df.groupby("PRODUCT_EXTRACTED")["CVE-ID"].count().sort_values(ascending=False)

In [65]:
final_scores = pd.DataFrame({
    "Total_Risk_Score": risk_by_product,
    "Nb_Vulnerabilities": count_by_product
})

# Trier par Total_Risk_Score décroissant
final_scores = final_scores.sort_values(by="Total_Risk_Score", ascending=False)

# Afficher le top 20 produits
print(final_scores.head(20))

                                        Total_Risk_Score  Nb_Vulnerabilities
PRODUCT_EXTRACTED                                                           
of Apache OFBiz                                      4.0                   1
Apache bRPC                                          4.0                   1
wordpress plugin Membership                          4.0                   1
wordpress plugin dukapress                           4.0                   1
wordpress plugin eventr                              4.0                   1
wordpress plugin flickr-picture-backup               4.0                   1
mod_sed of Apache                                    4.0                   1
ApacheTriad attackers to                             4.0                   1
Apache CXF attackers                                 4.0                   1
Apache OFBiz This                                    4.0                   1
Apache OFBiz an                                      4.0                   1

In [66]:
df = df[df["PRODUCT_EXTRACTED"].str.len() > 2]
df = df[~df["PRODUCT_EXTRACTED"].str.contains(r"^(An|The|A|User|Attacker|Publishing|Observable|Multi|Business|Engineering)$", regex=True)]


C:\Users\Zbook\AppData\Local\Temp\ipykernel_12996\4171658309.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df["PRODUCT_EXTRACTED"].str.contains(r"^(An|The|A|User|Attacker|Publishing|Observable|Multi|Business|Engineering)$", regex=True)]


In [67]:
df = df[~df["PRODUCT_EXTRACTED"].str.contains(r"\b(sending|uses|could|does|omission|unauthenticated)\b", case=False)]


C:\Users\Zbook\AppData\Local\Temp\ipykernel_12996\2560144200.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df["PRODUCT_EXTRACTED"].str.contains(r"\b(sending|uses|could|does|omission|unauthenticated)\b", case=False)]


In [68]:
valid_keywords = [
    "Windows", "Linux", "BIND", "SSH", "FTP", "TCP", "TCP/IP",
    "NFS", "IMAP", "POP", "PKCS", "statd", "rpc", "CDE", "Teardrop",
    "Chrome", "WordPress", "Cisco", "Apache", "Oracle"
]

df = df[df["PRODUCT_EXTRACTED"].str.contains("|".join(valid_keywords), case=False)]


In [69]:
risk_by_product = df.groupby("PRODUCT_EXTRACTED")["Risk_Score"].sum()
count_by_product = df.groupby("PRODUCT_EXTRACTED")["CVE-ID"].count()

final_scores = pd.DataFrame({
    "Total_Risk_Score": risk_by_product,
    "Nb_Vulnerabilities": count_by_product
}).sort_values(by="Total_Risk_Score", ascending=False)

print(final_scores.head(20))


                                        Total_Risk_Score  Nb_Vulnerabilities
PRODUCT_EXTRACTED                                                           
of Apache OFBiz                                      4.0                   1
Apache bRPC                                          4.0                   1
wordpress plugin Membership                          4.0                   1
wordpress plugin dukapress                           4.0                   1
wordpress plugin eventr                              4.0                   1
wordpress plugin flickr-picture-backup               4.0                   1
mod_sed of Apache                                    4.0                   1
ApacheTriad attackers to                             4.0                   1
Apache CXF attackers                                 4.0                   1
Apache OFBiz This                                    4.0                   1
Apache OFBiz an                                      4.0                   1

In [70]:
# Supprimer les lignes contenant des noms propres typiques (prénom + nom)
df = df[~df["PRODUCT_EXTRACTED"].str.match(r"^[A-Z][a-z]+\s[A-Z][a-z]+")]

# Supprimer lignes contenant des mots non techniques
df = df[~df["PRODUCT_EXTRACTED"].str.contains(r"\b(for|when|during|before|after|x|SQL)\b", case=False)]

# Garder uniquement les produits avec max 3 mots
df = df[df["PRODUCT_EXTRACTED"].str.split().str.len() <= 3]

# Supprimer les doublons
df["PRODUCT_EXTRACTED"] = df["PRODUCT_EXTRACTED"].str.strip()
df = df[df["PRODUCT_EXTRACTED"] != ""]
df = df.drop_duplicates(subset=["PRODUCT_EXTRACTED"])


C:\Users\Zbook\AppData\Local\Temp\ipykernel_12996\1591521512.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df["PRODUCT_EXTRACTED"].str.contains(r"\b(for|when|during|before|after|x|SQL)\b", case=False)]


In [71]:
risk_by_product = df.groupby("PRODUCT_EXTRACTED")["Risk_Score"].sum()
count_by_product = df.groupby("PRODUCT_EXTRACTED")["CVE-ID"].count()

final_scores = pd.DataFrame({
    "Total_Risk_Score": risk_by_product,
    "Nb_Vulnerabilities": count_by_product
}).sort_values(by="Total_Risk_Score", ascending=False)

print(final_scores.head(20))


                                        Total_Risk_Score  Nb_Vulnerabilities
PRODUCT_EXTRACTED                                                           
of Apache OFBiz                                      4.0                   1
Apache bRPC                                          4.0                   1
wordpress plugin Membership                          4.0                   1
wordpress plugin dukapress                           4.0                   1
wordpress plugin eventr                              4.0                   1
wordpress plugin flickr-picture-backup               4.0                   1
mod_sed of Apache                                    4.0                   1
ApacheTriad attackers to                             4.0                   1
Apache CXF attackers                                 4.0                   1
Apache OFBiz This                                    4.0                   1
Apache OFBiz an                                      4.0                   1

In [72]:
# Liste de produits valides
valid_products = ["Apache", "WordPress", "Linux", "Windows", "BIND", "TCP/IP", "FTP", 
                  "IMAP", "POP", "SSH", "CDE", "Teardrop", "Land", "PKCS", "statd", "rpc"]

# Fonction pour nettoyer le champ PRODUCT_EXTRACTED
def clean_product(prod):
    if not isinstance(prod, str):
        return None
    # Supprimer mots inutiles
    prod = prod.strip()
    prod = prod.replace("of ", "").replace("in ", "").replace("This", "").replace("an ", "").replace("to", "")
    prod = prod.replace("attackers", "").replace("plugin", "").replace("a", "").strip()
    # Garder seulement le premier mot si c’est un produit valide
    for p in valid_products:
        if p.lower() in prod.lower():
            return p
    return None

# Appliquer le nettoyage
df["PRODUCT_CLEAN"] = df["PRODUCT_EXTRACTED"].apply(clean_product)

# Supprimer les lignes sans produit valide
df_clean = df.dropna(subset=["PRODUCT_CLEAN"])

# Recalculer le scoring
risk_by_product = df_clean.groupby("PRODUCT_CLEAN")["Risk_Score"].sum()
count_by_product = df_clean.groupby("PRODUCT_CLEAN")["CVE-ID"].count()

final_scores = pd.DataFrame({
    "Total_Risk_Score": risk_by_product,
    "Nb_Vulnerabilities": count_by_product
}).sort_values(by="Total_Risk_Score", ascending=False)

print(final_scores.head(20))


               Total_Risk_Score  Nb_Vulnerabilities
PRODUCT_CLEAN                                      
WordPress                 498.0                 198
Linux                     486.0                 204
rpc                       402.0                 146
CDE                       113.0                  43
POP                         4.0                   1
IMAP                        3.0                   1
BIND                        3.0                   1
FTP                         3.0                   1
Windows                     3.0                   1
SSH                         3.0                   1
PKCS                        2.0                   1
TCP/IP                      2.0                   1


In [73]:
final_scores["Avg_Risk_Per_Vuln"] = final_scores["Total_Risk_Score"] / final_scores["Nb_Vulnerabilities"]
print(final_scores.head(20))

               Total_Risk_Score  Nb_Vulnerabilities  Avg_Risk_Per_Vuln
PRODUCT_CLEAN                                                         
WordPress                 498.0                 198           2.515152
Linux                     486.0                 204           2.382353
rpc                       402.0                 146           2.753425
CDE                       113.0                  43           2.627907
POP                         4.0                   1           4.000000
IMAP                        3.0                   1           3.000000
BIND                        3.0                   1           3.000000
FTP                         3.0                   1           3.000000
Windows                     3.0                   1           3.000000
SSH                         3.0                   1           3.000000
PKCS                        2.0                   1           2.000000
TCP/IP                      2.0                   1           2.000000


In [75]:
final_scores.to_csv("Cyber_Risk_Scoring_by_Product.csv", index_label="PRODUCT")